# Python Exercises

This exercise uses weekly data in Apple from the  rst week of January,
2010 to the last week of January, 2021. Use the adjusted closing prices
to compute weekly cc returns. Assume that the returns are i.i.d., even
though there may be some autocorrelation and volatility clustering is
likely. In bootstrap, let B = 1000 and set the random seed to 432, i.e.,
np.random.seed(432) in Python.

In [2]:
import pandas_datareader as web
import numpy as np
from scipy.stats import iqr, norm

In [3]:
df = web.get_data_yahoo("AAPL", start = "2010-01-01", end = "2021-01-31", interval='wk')
df.reset_index(inplace = True) # convert index into a column

Ret = np.log(df['Adj Close']/df['Adj Close'].shift(1))
Ret = np.array(Ret)[1:]
T = len(Ret)

(a) Find the **sample mean** and the **sample variance** of the cc return,
and also their **standard errors (SEs)** using the formula provided
in the slides of statistical inference.

In [4]:
mu_h    = np.mean(Ret) #sample mean
sig2_h  = np.var(Ret, ddof = 1)  #sample variance
sd_mu   = np.sqrt(sig2_h/T) #standard error of mu_h
sd_sig2 = sig2_h * np.sqrt(2/T) #standard error of sig2_h

print("The sample mean is %.6f" % mu_h)
print("The sample variance is %.6f" % sig2_h)
print("The standard error of mu_hat is %.6f" % sd_mu)
print("The standard error of sig2_hat is %.6f" % sd_sig2)

The sample mean is 0.005240
The sample variance is 0.001373
The standard error of mu_hat is 0.001541
The standard error of sig2_hat is 0.000081


(b)
- Find the bootstrap SEs (the **bootstrap SEs** and the **bootstrap IQR SEs**) of the **sample mean** and the **sample variance**.
- Compare the bootstrap SEs with the SEs in (a). Are they very different from each other for the same estimator (i.e., the sample mean and the sample variance)?

In [5]:
np.random.seed(432)
B = 1000
boot_samples = np.zeros((T, B))
for b in range(B):
    # take a random sample each iteration
    boot_samples[:,b] = np.random.choice(Ret, replace = True, size = T) 
    
boot_means = np.mean(boot_samples, axis = 0) # calculate sample mean for each column/iteration
B_SE_mu = np.std(boot_means, ddof = 1)
IQR_SE_mu = iqr(boot_means, interpolation = 'linear')/(norm.ppf(0.75) - norm.ppf(0.25))
print("The bootstrap SE of mu_hat is %.6f" % B_SE_mu)
print("The IQR SE of mu_hat is %.6f" % IQR_SE_mu)

boot_vars = np.var(boot_samples, ddof = 1, axis = 0) # calculate sample variance for each column/iteration
B_SE_sig2 = np.std(boot_vars, ddof = 1)
IQR_SE_sig2 = iqr(boot_vars, interpolation = 'linear')/(norm.ppf(0.75) - norm.ppf(0.25))
print("The bootstrap SE of sig2_hat is %.6f" % B_SE_sig2)
print("The IQR SE of sig2_hat is %.6f" % IQR_SE_sig2)

The bootstrap SE of mu_hat is 0.001556
The IQR SE of mu_hat is 0.001567
The bootstrap SE of sig2_hat is 0.000114
The IQR SE of sig2_hat is 0.000109


In General, the respective values are close to each other which implies that the bootstrap method is usefule in estimating the characteristics of the real/sample population:
- The bootstrap SE of mu_hat is 0.001556 and a little higher than the sample mean SE of 0.001541.
- The bootstrap SE of sig2_hat is 0.000114 and also a little higher the sample variance SE of sig2_hat of 0.000081.

(c) Find the bootstrap SEs (the **bootstrap SEs** and the **bootstrap IQR SEs**) of the sample standard deviation.

In [6]:
boot_sds = np.std(boot_samples, ddof = 1, axis = 0)
B_SE_sig = np.std(boot_sds, ddof = 1)
IQR_SE_sig = iqr(boot_sds, interpolation = 'linear')/(norm.ppf(0.75) - norm.ppf(0.25))
print("The bootstrap SE of sig_hat is %.6f" % B_SE_sig)
print("The IQR SE of sig_hat is %.6f" % IQR_SE_sig)

The bootstrap SE of sig_hat is 0.001542
The IQR SE of sig_hat is 0.001479


(d) Estimate the bias and the mean square error of the sample standard deviation using bootstrap.

In [7]:
sig_h = np.std(Ret, ddof = 1)
B_Bias_sig = np.mean(boot_sds) - sig_h
B_MSE_sig = np.mean((boot_sds - sig_h) ** 2)
print("The bootstrap bias of sig_hat is %.6f" % B_Bias_sig)
print("The bootstrap MSE of sig_hat is %.6f" % B_MSE_sig)

The bootstrap bias of sig_hat is -0.000141
The bootstrap MSE of sig_hat is 0.000002


(e) Construct the equal tail and the symmetric 95% bootstrap confidence intervals of the standard deviation of the cc return.

In [8]:
q_et_1 = np.quantile(sig_h - boot_sds, 0.025)
q_et_2 = np.quantile(sig_h - boot_sds, 0.975)
q_sym = np.quantile(np.abs(boot_sds - sig_h ), 0.95)

print([sig_h + q_et_1, sig_h + q_et_2])
print([sig_h - q_sym, sig_h + q_sym])

[0.03413049198104888, 0.040135445834804995]
[0.0340698466906352, 0.040030283556042226]


(f) Suppose we invest 1000 USD on Apple for one week. Find the **parametric** and the **nonparametric** estimators of **VaR(0.1)** and **ES(0.1)** for this investment. For the parametric estimator you shall assume that the returns are normally distributed.

In [9]:
W0 = 1000
q1 = norm.ppf(0.1, loc = mu_h, scale = sig_h)
q2 = np.quantile(Ret, 0.1, interpolation='linear')

VaR_1 = W0 * (np.exp(q1) - 1)
VaR_2 = W0 * (np.exp(q2) - 1)
print("The parametric estimate of VaR(0.1) is %.6f" % VaR_1)
print("The nonparametric estimate of VaR(0.1) is %.6f" % VaR_2)

nsim = 500000
r_B = norm.rvs(loc = mu_h, scale = sig_h, size=nsim, random_state=432)
L_B = W0 * (np.exp(r_B) - 1)
I_B = (L_B <= VaR_1) * 1
ES_1 = np.mean(L_B * I_B)/np.mean(I_B)

L_1 = W0 * (np.exp(Ret) - 1)
I_1 = (L_1 <= VaR_2)
ES_2 = np.mean(L_1 * I_1)/np.mean(I_1)

print("The parametric estimate of ES(0.1) is %.6f" % ES_1)
print("The nonparametric estimate of ES(0.1) is %.6f" % ES_2)

The parametric estimate of VaR(0.1) is -41.361413
The nonparametric estimate of VaR(0.1) is -38.195320
The parametric estimate of ES(0.1) is -57.830475
The nonparametric estimate of ES(0.1) is -62.535312


(g) Find the bootstrap SEs (the **bootstrap SEs** and the **bootstrap IQR
SEs**) of the nonparametric estimators of VaR(0.1) and ES(0.1) in
(f).

In [11]:
# Inference on VaR based on bootstrap
def VaR(y, p = 0.1, W0 = W0):
    mu = np.mean(y)
    q = np.quantile(y, p, interpolation='linear')
    return W0 * (np.exp(q) - 1)

# Inference on ES based on bootstrap
def ES(y, p=0.1, W0 = W0):
    mu = np.mean(y)
    q = np.quantile(y, p, interpolation='linear')
    VaR_np = W0 * (np.exp(q) - 1)
    L_1 = W0 * (np.exp(y) - 1)
    I_1 = (L_1 <= VaR_np) * 1
    return np.mean(L_1 * I_1)/np.mean(I_1)

    
def boot(data, func, nboot=5000, n=T):
    """
    func: function, VaR or ES
    nboot: number of bootstrap samples
    n: size of a boostrap sample
    """
    result = np.zeros(nboot)
    for i in range(nboot):
        boot_sample = np.random.choice(data, replace = True, size = n)
        result[i] = func(boot_sample)
    return result

np.random.seed(432)

VaR_est = VaR(Ret)

boot_VaR = boot(data = Ret, func = VaR)
BootSE_VaR = np.std(boot_VaR, ddof = 1)
Boot_SE_CI095_VaR = [VaR_est - BootSE_VaR*1.96, VaR_est + BootSE_VaR*1.96] # Bootstrap CI using Boot-SE

Boot_IQR_SE_VaR = (np.quantile(boot_VaR, 0.75) - np.quantile(boot_VaR, 0.25))/(norm.ppf(0.75) - norm.ppf(0.25))
Boot_IQR_SE_CI095_VaR = [VaR_est - Boot_IQR_SE_VaR * 1.96, VaR_est + Boot_IQR_SE_VaR * 1.96] # Bootstrap CI using IQR Boot-SE

print("The bootstrap SE of the nonparametric estimator of VaR(0.1) is %.6f" % BootSE_VaR)
print("The bootstrap IQR SE of the nonparametric estimator of VaR(0.1) is %.6f" % Boot_IQR_SE_VaR)


ES_est = ES(Ret)

boot_ES = boot(data = Ret, func = ES)
BootSE_ES = np.std(boot_ES, ddof = 1)
Boot_SE_CI095_ES = [ES_est - BootSE_ES*1.96, ES_est + BootSE_ES*1.96] # Bootstrap CI using Boot-SE
 

Boot_IQR_SE_ES = (np.quantile(boot_ES, 0.75) - np.quantile(boot_ES, 0.25))/(norm.ppf(0.75) - norm.ppf(0.25))
Boot_IQR_SE_CI095_ES = [ES_est - Boot_IQR_SE_ES * 1.96, ES_est + Boot_IQR_SE_ES * 1.96] # Bootstrap CI using IQR Boot-SE

print("The bootstrap SE of the nonparametric estimator of ES(0.1) is %.6f" % BootSE_ES)
print("The bootstrap IQR SE of the nonparametric estimator of ES(0.1) is %.6f" % Boot_IQR_SE_ES)

The bootstrap SE of the nonparametric estimator of VaR(0.1) is 2.743492
The bootstrap IQR SE of the nonparametric estimator of VaR(0.1) is 2.708308
The bootstrap SE of the nonparametric estimator of ES(0.1) is 4.433590
The bootstrap IQR SE of the nonparametric estimator of ES(0.1) is 4.437280


(h) Find the 95% confidence intervals of VaR(0.1) and ES(0.1) using
the nonparametric method in (f).

In [12]:
print("The 95 percent CI using the bootstrap SE is ", Boot_SE_CI095_VaR)
print("The 95 percent CI using the bootstrap IQR SE is ", Boot_IQR_SE_CI095_VaR)
print("The 95 percent CI using the bootstrap SE is ", Boot_SE_CI095_ES)
print("The 95 percent CI using the bootstrap IQR SE is ", Boot_IQR_SE_CI095_ES)

The 95 percent CI using the bootstrap SE is  [-43.57256419619579, -32.81807642287951]
The 95 percent CI using the bootstrap IQR SE is  [-43.503603521077856, -32.88703709799744]
The 95 percent CI using the bootstrap SE is  [-71.22514853676557, -53.845475275009754]
The 95 percent CI using the bootstrap IQR SE is  [-71.23238063753166, -53.83824317424366]
